# Setup Instructions 

### (1) Create lcls-live environment ( which includes conda-forged version of Bamd and Tao )

####    (a) Copy https://github.com/ChristopherMayes/lcls-live/blob/master/environment.yml
####    (b) conda env create -f environment.yml
####    (c) source activate lcls-live
####    (d) python -m ipykernel install --user --name lcls-live --display-name Lcls-live


### (2) Gain access to lcls-lattice

#### If on personal computer, `git clone https://github.com/slaclab/lcls-lattice.git`
#### If on SLAC SDF, use Chris’s directory: `export LCLS_LATTICE=/u/ad/cmayes/nfs/GitHub/lcls-lattice/`
#### For details go to https://slaclab.github.io/lcls-live/getting-started.html


# Codes

In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = 'retina'

# Initialize Tao

In [1]:
#from lcls_live.bmad import LCLSTaoModel
#from lcls_live.epics import epics_proxy

import os

from pytao import Tao
import pandas as pd

In [2]:
# Basic model with options

#MODEL = 'sc_sxr' # For SPS beamline
#MODEL = 'sc_hxr' # For SPH beamline
MODEL = 'sc_bsyd' # For SPD beamline


OPTIONS = f'-noplot'
INIT = f'-init $LCLS_LATTICE/bmad/models/{MODEL}/tao.init {OPTIONS}'

# Make sure this exists
assert 'LCLS_LATTICE' in os.environ

#INIT

In [3]:
# Do NOT run this cell twice within one kernel instance
tao = Tao(INIT)

In [6]:
# Make sure Bmad version is up-to-date
# tao.so_lib

# BPM list

In [4]:
# Set the beginning and end elements between which BPMs are to be found

ele_start = 'BEGGUNB'

#ele_end   = 'ENDSPS'  # for SPS beamline
#ele_end   = 'ENDSPH'  # for SPH beamline
ele_end   = 'ENDSLTD'  # for SPD beamline

In [5]:
# Check if the elements specified exist
if len(tao.lat_list(ele_end,'ele.ix_ele')) == 0:
    print('The end element specified is not in the current lattice!!')

In [6]:
ix_start = tao.lat_list(ele_start,'ele.ix_ele')[0]
ix_end   = tao.lat_list(ele_end,'ele.ix_ele')[0]

BPM_string = f'{ix_start}:{ix_end}&BPM*'

name_list  = tao.lat_list(BPM_string,'ele.name')
s_list     = tao.lat_list(BPM_string,'ele.s')
eta_x_list = tao.lat_list(BPM_string,'ele.eta_x')

In [7]:
df=pd.DataFrame()
df['Name'] = name_list 
df['s(m)'] = s_list
df['eta(m)'] = eta_x_list

In [8]:
# Drop the BPMs with "non-zero" dispersion
df2 = df.drop(df[abs(df['eta(m)']) > 1e-6].index)
df2

,Name,s(m),eta(m)
0,BPM1B,0.489650,0.000000e+00
1,BPM2B,1.884224,0.000000e+00
2,BPM0H01,18.645042,0.000000e+00
3,BPM0H04,24.035167,0.000000e+00
4,BPM0H05,25.257467,0.000000e+00
...,...,...,...
85,BPMDAS,2934.015161,-1.831367e-13
86,BPMSP2D,2972.290761,-5.198622e-13
87,BPMSP3D,3051.005727,-7.017107e-13
88,BPMSP4D,3157.709420,-9.005216e-13


In [9]:
# Reset index if needed
df2.reset_index(drop=True)

,Name,s(m),eta(m)
0,BPM1B,0.489650,0.000000e+00
1,BPM2B,1.884224,0.000000e+00
2,BPM0H01,18.645042,0.000000e+00
3,BPM0H04,24.035167,0.000000e+00
4,BPM0H05,25.257467,0.000000e+00
...,...,...,...
73,BPMDAS,2934.015161,-1.831367e-13
74,BPMSP2D,2972.290761,-5.198622e-13
75,BPMSP3D,3051.005727,-7.017107e-13
76,BPMSP4D,3157.709420,-9.005216e-13


In [15]:
# Save the file as a CSV
df2.to_csv('file1.csv')